## Laboratorio práctico: Construir un panel interactivo con Plotly Dash
En este laboratorio, construirás una aplicación de Plotly Dash para que los usuarios realicen análisis visuales interactivos sobre los datos de lanzamientos de SpaceX en tiempo real.

Esta aplicación de panel contiene componentes de entrada como una lista desplegable y un control deslizante de rango para interactuar con un gráfico circular y un gráfico de dispersión. Serás guiado para construir esta aplicación de panel a través de las siguientes tareas:

- TAREA 1: Agregar un componente de entrada desplegable de sitio de lanzamiento
- TAREA 2: Agregar una función de callback para renderizar success-pie-chart basado en el sitio seleccionado en la lista desplegable
- TAREA 3: Agregar un control deslizante de rango para seleccionar la carga útil
- TAREA 4: Agregar una función de callback para renderizar el gráfico de dispersión success-payload-scatter-chart

In [ ]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Carga del dataset
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Crear la aplicación Dash
app = dash.Dash(__name__)

# Layout de la app con los componentes pedidos
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    
    # TAREA 1: Dropdown para seleccionar sitio de lanzamiento
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'},
            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
        ],
        value='ALL',  # Valor por defecto para seleccionar todos los sitios
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    html.Br(),

    # TAREA 2: Gráfico circular (pie chart) para mostrar éxitos
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    html.P("Payload range (Kg):"),

    # TAREA 3: Slider para seleccionar rango de carga útil
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
        value=[min_payload, max_payload]  # Rango por defecto con todos los valores
    ),

    # TAREA 4: Gráfico de dispersión para carga útil vs éxito
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# Callback para actualizar el gráfico de pastel (pie chart)
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        # Total de lanzamientos exitosos por sitio para todos los sitios
        filtered_df = spacex_df[spacex_df['class'] == 1]
        fig = px.pie(filtered_df,
                     names='Launch Site',
                     title='Total Successful Launches by Site')
    else:
        # Para un sitio específico, mostrar éxitos y fallos
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        counts = filtered_df['class'].value_counts().reset_index()
        counts.columns = ['class', 'count']
        counts['class'] = counts['class'].replace({1: 'Success', 0: 'Failure'})
        fig = px.pie(counts,
                     names='class',
                     values='count',
                     title=f'Success vs Failure Launches for site {selected_site}')
    return fig

# Callback para actualizar el gráfico de dispersión (scatter chart)
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [
        Input(component_id='site-dropdown', component_property='value'),
        Input(component_id='payload-slider', component_property='value')
    ]
)
def update_scatter_chart(selected_site, payload_range):
    low, high = payload_range
    # Filtrar por rango de carga útil
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) &
                            (spacex_df['Payload Mass (kg)'] <= high)]
    if selected_site != 'ALL':
        # Filtrar por sitio de lanzamiento
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]

    fig = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',
        title='Correlation between Payload and Success for Launch Site(s)',
        labels={'class': 'Launch Outcome (0=Failure, 1=Success)'},
        hover_data=['Launch Site']
    )
    return fig

# Ejecutar la app
if __name__ == '__main__':
    app.run(debug=True)
